#   K-Nearest Neighbor 

Δε χρησιμοποιήθηλε κάποια βιβλιοθήκη της sklearn , η πηγή του κώδικα μου αναφέρεται παρακάτω αλλά την βάζω και εδώ
#https://medium.com/analytics-vidhya/implementing-k-nearest-neighbours-knn-without-using-scikit-learn-3905b4decc3c .
Δεν έγινε η roc_plot , επίσης στο πρώτο 10 k-fold, το έκανα μόνο για 5 γιατί αργούσε πολύ δυστυχώς στο δικό μου 
υπολογιστή(3λεπτά το κάθε τρέξιμο). Το Bonus όπως και στις άλλες έτσι και εδώ βρίσκεται προς το τέλος του 
notebook.

In [1]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
import numpy as np
import scipy
from math import sqrt
from collections import Counter

In [2]:
#eisagw ta dedomena gia sugkrisi
data = pd.read_csv('data.tsv',sep="\t")
X = data["content"]
#allazw ti morfi twn category apo business,tech,politics klp se ari8mous apo to 0-4
le = preprocessing.LabelEncoder()
le.fit(data["category"])
y = le.transform(data["category"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
#source
#https://medium.com/analytics-vidhya/implementing-k-nearest-neighbours-knn-without-using-scikit-learn-3905b4decc3c
class KNN:
    def __init__(self, k = 3): #an de do8ei k stin arxikopoiisi tis KNN tote pairnei ws default timi to 5
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
    def euclidean_distance(self, instance1, instance2):
        # just in case, if the instances are lists or tuples:
        instance1 = np.array(instance1) 
        instance2 = np.array(instance2)
        return np.linalg.norm(instance1 - instance2)
    
    def predict(self, X_test):
        final_output = []
        for i in range(len(X_test)):
            d = []
            votes = []
            for j in range(len(self.X_train)):
                dist = self.euclidean_distance(self.X_train[j] , X_test[i])
                d.append([dist, j])
            d.sort()
            d = d[0:self.k]
            for d, j in d:
                votes.append(self.y_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            final_output.append(ans)
            
        return final_output

In [5]:
pipeline = Pipeline([ ('vect', CountVectorizer()),('tfidf', TfidfTransformer()) ,
                     ('lsi', TruncatedSVD(n_components = 50)) ,('clf', KNN())])

In [6]:
#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test)

precision = precision_score(y_test, y_pred, average = 'macro')
recall = recall_score(y_test,y_pred,average = 'macro')
f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for Random Forests(RF):'),
print('Precision:',precision,'\nRecall:',recall,'\nF-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for Random Forests(RF):
Precision: 0.8157087248714809 
Recall: 0.8162012727467379 
F-Measure: 0.8155122933014958 
Accuracy: 0.8186504217432052


In [7]:
precision = 0
recall = 0
f1 = 0
accuracy = 0

ss = ShuffleSplit(n_splits=10, test_size=0.20, random_state=42)
#10-fold cross validation
for train,test in ss.split(X, y):
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

    #Train the classifier with the training split
    pipeline.fit(X_train,y_train)

    #Predict the test split
    y_pred = pipeline.predict(X_test)
    
    precision += precision_score(y_test, y_pred, average = 'macro')
    recall += recall_score(y_test,y_pred,average = 'macro')
    f1 += f1_score(y_test,y_pred,average = 'macro')
    accuracy += accuracy_score(y_test, y_pred)
    
precision = precision/10
recall = recall/10
f1 = f1/10
accuracy = accuracy/10
    
print('Cross Validation results for Random Forests(RF):'),
print('Precision:',precision,'\nRecall:',recall,'\nF-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for Random Forests(RF):
Precision: 0.800727396331762 
Recall: 0.8013054806241557 
F-Measure: 0.7998195946072888 
Accuracy: 0.8025304592314901


# Bonus

In [37]:
#Bonus K_Nearest_Neighbor
#exw ftiaksei ena updated_dataset pou to exw ka8arisei apo perittes le3eis kai testarw gia auto tous 
#algori8mous gia na dw poso kaluteri einai i epidosi tous

#eisagw ta dedomena gia sugkrisi
data = pd.read_csv('updated_data.tsv',sep="\t")
X = data["content"]
#allazw ti morfi twn category apo business,tech,politics klp se ari8mous apo to 0-4
le = preprocessing.LabelEncoder()
le.fit(data["category"])
y = le.transform(data["category"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
pipeline = Pipeline([ ('vect', CountVectorizer()),('tfidf', TfidfTransformer()) ,
                     ('lsi', TruncatedSVD(n_components = 50)) ,('clf', KNN())])

In [39]:
#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train.values.astype(str),y_train)

#Predict the test split
y_pred = pipeline.predict(X_test.values.astype(str))

precision = precision_score(y_test, y_pred, average = 'macro')
recall = recall_score(y_test,y_pred,average = 'macro')
f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for Random Forests(RF):'),
print('Precision:',precision,'\nRecall:',recall,'\nF-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for Random Forests(RF):
Precision: 0.8758401266662086 
Recall: 0.8762572785080991 
F-Measure: 0.8755005651713773 
Accuracy: 0.8781630740393627


In [40]:
precision = 0
recall = 0
f1 = 0
accuracy = 0

ss = ShuffleSplit(n_splits=10, test_size=0.20, random_state=0)
#10-fold cross validation
for train,test in ss.split(X, y):
    X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

    #Train the classifier with the training split
    pipeline.fit(X_train.values.astype(str),y_train)

    #Predict the test split
    y_pred = pipeline.predict(X_test.values.astype(str))
    
    precision += precision_score(y_test, y_pred, average = 'macro')
    recall += recall_score(y_test,y_pred,average = 'macro')
    f1 += f1_score(y_test,y_pred,average = 'macro')
    accuracy += accuracy_score(y_test, y_pred)
    
precision = precision/10
recall = recall/10
f1 = f1/10
accuracy = accuracy/10
    
print('Cross Validation results for Random Forests(RF):'),
print('Precision:',precision,'\nRecall:',recall,'\nF-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for Random Forests(RF):
Precision: 0.8834446932430661 
Recall: 0.8847571791475708 
F-Measure: 0.8834490455551061 
Accuracy: 0.8852389878163074
